In [1]:
import pandas as pd
import numpy as np
import torch 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch import nn
from matplotlib import cm
import cv2

import PIL 
from PIL import Image as im
from IPython.display import Image 
import xmltodict as xd

# !pip install git+https://github.com/jlevy44/PathPretrain

import tifffile
from pathpretrain.utils import load_image
from pathpretrain.process_xml import process_xml

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
base_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/scc_nuclei/data/"

xml_files = [
    "350 A1b_ASAP.xml", 
    "355 A1d_ASAP.xml",    
]
annotations = {}

In [49]:
for f in tqdm(xml_files): 
    annotations[f] = process_xml(base_dir+f, transpose=False)['contour']

  0%|                                                                                                                                                                                 | 0/2 [00:00<?, ?it/s]/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][120]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[1][120]=x[1][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][14]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/l

/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][111]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[1][111]=x[1][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[0][160]=x[0][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anaconda3/envs/jupyter-ultimate/lib/python3.7/site-packages/scipy/interpolate/_fitpack_impl.py:227: RuntimeWarning: Setting x[1][160]=x[1][0]
  (i, m, i)))
/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Gokul_Srinivasan/anacon

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.70it/s]


In [50]:
pd_350 = annotations["350 A1b_ASAP.xml"]


In [51]:
pd_350

,contours,xmin,xmax,ymin,ymax,xmean,ymean,lbl
0,"[[16352.320299999996, 13668.3926], [16352.6556...",14545.5791,23631.6563,9387.20020,13890.9619,20063.232000,11699.087912,scc
1,"[[22165.3145, 13354.175799999999], [22164.1440...",22165.3145,23291.2559,12935.22170,13498.1924,22695.554407,13230.734664,scc
2,"[[22230.7773, 8876.599609999997], [22233.32551...",22008.2070,22937.7617,8876.59961,10421.4951,22431.525513,9768.623895,scc
3,"[[23382.9004, 10748.802699999998], [23386.4894...",22846.1152,23958.9648,9033.70801,10827.3564,23397.240610,9937.701912,scc
4,"[[16791.619099999996, 9931.884769999999], [167...",15858.7500,17766.5703,8914.84766,10422.8672,16584.982025,9575.726750,scc
...,...,...,...,...,...,...,...,...
103,"[[18006.642599999992, 72299.23439999997], [180...",17897.5742,18107.0977,72292.53910,72497.2734,18004.368656,72394.845719,scc
104,"[[17551.912100000005, 49350.234400000016], [17...",17049.8965,18138.7305,49318.01170,50235.5469,17568.270578,49816.446791,inflammation
105,"[[17714.1582, 43860.2773], [17711.759390603653...",16829.5156,17738.7305,43860.27340,45106.9648,17331.410716,44418.390500,inflammation
106,"[[15447.7939, 13603.918], [15447.376425310753,...",14473.0410,15453.3535,13222.16990,13817.0283,14958.180966,13516.180666,inflammation


In [52]:
new_pd_350 = pd.DataFrame(columns=["contours", "xmin", "xmax", "ymin", "ymax", "xmean", "ymean", "lbl"])

In [53]:
contours = []
xmin = []
xmax = []
ymin = []
ymax = []
xmean = []
ymean = []
lbl = []

In [54]:
for idx, row in pd_350.iterrows():
    #get all the new stats
    x_min = 100000000
    y_min = 100000000
    x_max = 0 
    y_max = 0 
    x_mean = []
    y_mean = []
#     print( row["contours"])
#     break
    for pair in row["contours"]:
        x = pair[0]
        y = pair[1]
        print(pair)
        
#         x_min = min(x_min, x)
#         x_max = max(x_max, x)
#         y_min = min(y_min, y)
#         y_max = max(y_max, y)
        
#         x_mean.append(x)
#         y_mean.append(y)
    break
    x_mean = sum(x_mean)/len(x_mean)
    y_mean = sum(y_mean)/len(y_mean)
    
    #add them to list 
    contours.append(row["contours"])
    xmin.append(x_min)
    xmax.append(x_max)
    ymin.append(y_min)
    ymax.append(y_max)
    xmean.append(x_mean)
    ymean.append(y_mean)
    lbl.append(row["lbl"])

new_pd_350["contours"] = contours
new_pd_350["xmin"] = xmin
new_pd_350["xmax"] = xmax
new_pd_350["ymin"] = ymin
new_pd_350["ymax"] = ymax
new_pd_350["xmean"] = xmean
new_pd_350["ymean"] = ymean
new_pd_350["lbl"] = lbl

[16352.3203 13668.3926]
[16352.65566202 13638.58383691]
[16354.24105806 13607.98228659]
[16356.40641965 13576.97019138]
[16358.48167837 13545.9297936 ]
[16359.79676576 13515.24333561]
[16359.68161339 13485.29305974]
[16357.46615281 13456.46120831]
[16352.48031558 13429.13002367]
[16344.05403325 13403.68174816]
[16331.56099404 13380.47173005]
[16315.10886957 13359.404191  ]
[16295.41384631 13340.00934254]
[16273.2104969  13321.80609555]
[16249.23339397 13304.31336089]
[16224.21711013 13287.05004946]
[16198.89621802 13269.53507211]
[16174.00529028 13251.28733974]
[16150.27889952 13231.82576322]
[16128.45161837 13210.66925342]
[16109.2444377 13187.3473962]
[16092.86422734 13161.79386479]
[16078.85791109 13134.46103502]
[16066.72909474 13105.8353297 ]
[16055.98138409 13076.40317159]
[16046.11838492 13046.65098351]
[16036.64370302 13017.06518823]
[16027.06094418 12988.13220855]
[16016.87371419 12960.33846726]
[16005.58561884 12934.17038715]
[15992.70026392 12910.11439102]
[15977.72125522 12

In [55]:
new_pd_350

,contours,xmin,xmax,ymin,ymax,xmean,ymean,lbl


In [56]:
pd_350

,contours,xmin,xmax,ymin,ymax,xmean,ymean,lbl
0,"[[16352.320299999996, 13668.3926], [16352.6556...",14545.5791,23631.6563,9387.20020,13890.9619,20063.232000,11699.087912,scc
1,"[[22165.3145, 13354.175799999999], [22164.1440...",22165.3145,23291.2559,12935.22170,13498.1924,22695.554407,13230.734664,scc
2,"[[22230.7773, 8876.599609999997], [22233.32551...",22008.2070,22937.7617,8876.59961,10421.4951,22431.525513,9768.623895,scc
3,"[[23382.9004, 10748.802699999998], [23386.4894...",22846.1152,23958.9648,9033.70801,10827.3564,23397.240610,9937.701912,scc
4,"[[16791.619099999996, 9931.884769999999], [167...",15858.7500,17766.5703,8914.84766,10422.8672,16584.982025,9575.726750,scc
...,...,...,...,...,...,...,...,...
103,"[[18006.642599999992, 72299.23439999997], [180...",17897.5742,18107.0977,72292.53910,72497.2734,18004.368656,72394.845719,scc
104,"[[17551.912100000005, 49350.234400000016], [17...",17049.8965,18138.7305,49318.01170,50235.5469,17568.270578,49816.446791,inflammation
105,"[[17714.1582, 43860.2773], [17711.759390603653...",16829.5156,17738.7305,43860.27340,45106.9648,17331.410716,44418.390500,inflammation
106,"[[15447.7939, 13603.918], [15447.376425310753,...",14473.0410,15453.3535,13222.16990,13817.0283,14958.180966,13516.180666,inflammation
